# Get a list of URLs for S&P 500 firms 

In [1]:
!pip install requests_html

In [2]:
import pandas as pd
from requests_html import HTMLSession # htmls is how it opens pages 
url='https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
session = HTMLSession()
r = session.get(url)
r.url

'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

In [3]:
r.html.links #gives me all the links on the webpage | all the realitive links 
r.html.absolute_links #clickable, dont have to stick together the links 

r.html.find('table')[0]
table = r.html.find('#constituents')[0]
table.text # prints the text 
table.html

#blunt force the table 
table.absolute_links
table.find('tr')[0].text

table_rows = table.find('tr')

colinks = []
for row in table_rows[1:]:
    colinks.append( list (row.find('td')[1].absolute_links)[0] )
colinks

    

['https://en.wikipedia.org/wiki/3M',
 'https://en.wikipedia.org/wiki/Abbott_Laboratories',
 'https://en.wikipedia.org/wiki/AbbVie_Inc.',
 'https://en.wikipedia.org/wiki/Abiomed',
 'https://en.wikipedia.org/wiki/Accenture_plc',
 'https://en.wikipedia.org/wiki/Activision_Blizzard',
 'https://en.wikipedia.org/wiki/Adobe_Inc.',
 'https://en.wikipedia.org/wiki/Advanced_Micro_Devices_Inc',
 'https://en.wikipedia.org/wiki/Advance_Auto_Parts',
 'https://en.wikipedia.org/wiki/AES_Corp',
 'https://en.wikipedia.org/wiki/AFLAC_Inc',
 'https://en.wikipedia.org/wiki/Agilent_Technologies_Inc',
 'https://en.wikipedia.org/wiki/Air_Products_%26_Chemicals_Inc',
 'https://en.wikipedia.org/wiki/Akamai_Technologies_Inc',
 'https://en.wikipedia.org/wiki/Alaska_Air_Group_Inc',
 'https://en.wikipedia.org/wiki/Albemarle_Corp',
 'https://en.wikipedia.org/wiki/Alexandria_Real_Estate_Equities',
 'https://en.wikipedia.org/wiki/Alexion_Pharmaceuticals',
 'https://en.wikipedia.org/wiki/Align_Technology',
 'https://en

# clean code above up 

In [4]:
import pandas as pd
from requests_html import HTMLSession 

url='https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
session = HTMLSession()
r = session.get(url)

table = r.html.find('#constituents')[0]

table_rows = table.find('tr')

colinks = []
for row in table_rows[1:]:
    colinks.append( list (row.find('td')[1].absolute_links)[0] )

colinks

['https://en.wikipedia.org/wiki/3M',
 'https://en.wikipedia.org/wiki/Abbott_Laboratories',
 'https://en.wikipedia.org/wiki/AbbVie_Inc.',
 'https://en.wikipedia.org/wiki/Abiomed',
 'https://en.wikipedia.org/wiki/Accenture_plc',
 'https://en.wikipedia.org/wiki/Activision_Blizzard',
 'https://en.wikipedia.org/wiki/Adobe_Inc.',
 'https://en.wikipedia.org/wiki/Advanced_Micro_Devices_Inc',
 'https://en.wikipedia.org/wiki/Advance_Auto_Parts',
 'https://en.wikipedia.org/wiki/AES_Corp',
 'https://en.wikipedia.org/wiki/AFLAC_Inc',
 'https://en.wikipedia.org/wiki/Agilent_Technologies_Inc',
 'https://en.wikipedia.org/wiki/Air_Products_%26_Chemicals_Inc',
 'https://en.wikipedia.org/wiki/Akamai_Technologies_Inc',
 'https://en.wikipedia.org/wiki/Alaska_Air_Group_Inc',
 'https://en.wikipedia.org/wiki/Albemarle_Corp',
 'https://en.wikipedia.org/wiki/Alexandria_Real_Estate_Equities',
 'https://en.wikipedia.org/wiki/Alexion_Pharmaceuticals',
 'https://en.wikipedia.org/wiki/Align_Technology',
 'https://en

**The Scrapper below is mimiced in Assignment 5**

In [5]:
import pandas as pd
from requests_html import HTMLSession
import os #lets pandas talk to the OS 
import re
from time import sleep
from tqdm.notebook import tqdm # import progress bar


#### SECTION 1: get the urls to crawl (or data that can build the urls)

if not os.path.exists('input/sp500_urls.csv'):

    # open page
    session = HTMLSession() 
    url = 'https://en.m.wikipedia.org/wiki/List_of_S%26P_500_companies'
    r = session.get(url)

    # find the links
    table = r.html.find('#constituents')[0]  # reduce to the table
    table_rows = table.find('tr')[1:]
    colinks = []
    for row in table_rows:
        a_url =   list(row.find('td')[1].absolute_links)[0]    
        colinks.append(a_url)

    # save list to csv    
    if not os.path.exists('input/'):
        os.makedirs('input/')
    pd.DataFrame(colinks).to_csv('input/sp500_urls.csv',index=False,header=False)
    
else:
    colinks = pd.read_csv('input/sp500_urls.csv',names=['firm']) ['firm'].to_list()


#### SECTION 2: crawl the urls

# for url in colinks[:10]: first 10 firms 
for url in tqdm(colinks,total=len(colinks)):
    
    folder = 'wiki_html/'
    if not os.path.exists('wiki_html/'):
        os.makedirs('wiki_html/')
    
    name = re.sub( r'\W+','',     url.split('/')[-1]    )
    filepath = folder + name + '.txt'
        
    if not os.path.exists(filepath):
        
        try:
            r = session.get(url)
        except:
            print('sad face failure:',url)
        else:   
            with open(filepath,'w',encoding='utf8') as f:
                f.write(r.text)
            sleep(2) # be nice to server        

